In [2]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "../datasets/scifact"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

/home/ab/.pyenv/versions/3.11.6/lib/python3.11/site-packages/beir/datasets/data_loader.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
100%|██████████| 5183/5183 [00:00<00:00, 23013.33it/s]


In [3]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "ab",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "JS2mz8c3RdOol8iLxCkFHA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost"
index_name = "scifact"
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

que: 100%|██████████| 3/3 [00:04<00:00,  1.38s/it]


In [5]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
ndcg, _map, recall, precision

({'NDCG@1': 0.57,
  'NDCG@3': 0.62578,
  'NDCG@5': 0.65712,
  'NDCG@10': 0.67764,
  'NDCG@100': 0.70067,
  'NDCG@1000': 0.70766},
 {'MAP@1': 0.54761,
  'MAP@3': 0.60377,
  'MAP@5': 0.62227,
  'MAP@10': 0.63245,
  'MAP@100': 0.6378,
  'MAP@1000': 0.63805},
 {'Recall@1': 0.54761,
  'Recall@3': 0.66678,
  'Recall@5': 0.74289,
  'Recall@10': 0.80044,
  'Recall@100': 0.90283,
  'Recall@1000': 0.9585},
 {'P@1': 0.57,
  'P@3': 0.23889,
  'P@5': 0.16133,
  'P@10': 0.089,
  'P@100': 0.01023,
  'P@1000': 0.00108})